# Imports

In [1]:
#export
import os
import sys
import atexit
sys.path.append(os.path.join(os.getcwd(),'exp'))

import fire
import docker
from functools import partial
from pathlib import Path

from config import cfg
from nb_locker import check_locks, list_locks, lock as locker

# Code

In [2]:
#export
def status():
    pattern = cfg.DOCKER.CONTAINER_PREFIX
    print(f'  *{pattern}* containers: ')
    
    for c in cycle_c_gen():
        gpu = check_gpu(c)
        print(f'\t{c.name} @ GPU{gpu} @ {c.status}')
    
    print(f'Active locks @ {cfg.GPUS.LOCK}:')
    lp = Path(cfg.GPUS.LOCK)
    list_locks(lp)
    
def cycle_c_gen(pat=cfg.DOCKER.CONTAINER_PREFIX, list_all=False):
    client = docker.from_env()
    containers = client.containers.list(all=list_all)
    for i, c in enumerate(containers):
        print(f'{i}. container\t {c.name} :')
        if pat in c.name:
            yield c
        else:
            print(f'\t Skipping {c.name}, not part of dHPO\n')
    
def check_gpu(container):
    env_vars = container.attrs['Config']['Env']
    g = None
    for v in env_vars:
        if v.startswith('CUDA_VISIBLE_DEVICES'):
            g = v.split("=")[-1]
    return g.split(',')

In [3]:
#export
def check_status(c):
    c.reload()
    return c.status

def is_running(c): return check_status(c) == 'running'
def is_paused(c): return check_status(c) == 'paused'

def unpause(c):
    if is_paused(c):
        print(f'\t UNpausing {c.name}...')
        c.unpause()
        print('\tSUCCESS')
        return True
    elif is_running(c):
        print(f'\tWARNING {c.name} already running!')
        return True
    else:
        print(f'\tSomething is wrong with {c.name}, check it manually')
        return False

def pause(c):
    if is_running(c):
        print(f'\t Pausing {c.name}...')
        c.pause()
        print('\tSUCCESS')
        return True
    elif is_paused(c):
        print(f'\tWARNING {c.name} already on pause!')
        return True
    else:
        print(f'\tSomething is wrong with {c.name}, check it manually')
        return False
    
def switch(gpus, mode):
    usage =''' Usage:
        switch $GPUS $MODE
        switch 0 pause
        switch 0,1 unpause
    '''
    if gpus is None or mode not in ['pause', 'unpause']:
        print(usage)
        return
        #exit(0)
    
    if not isinstance(gpus, tuple):
        gpus = gpus,
    gpus = set([str(i) for i in gpus])
    #print(gpus)
    if mode == 'pause':
        do = pause
    elif mode == 'unpause':
        do = unpause

    for c in cycle_c_gen():
        c_gpus = set(check_gpu(c))
        if c_gpus.intersection(gpus):
            print(f'\t{c.name} is using GPUS{c_gpus}, trying to set on "{mode}"')
            do(c)
        else:
            print(f'\tSkipping {c.name}, on GPU{c_gpus}')

def forced(func):
    def force(force_arg, *args, **kwargs):
        if force_arg == 'force':
            return func(*args, **kwargs)
        else:
            print('Specify force arg: dhpoctl foo force')
    return force
            
@forced           
def clean():
        for c in cycle_c_gen(list_all=True):
            if c.status == 'exited':
                c.remove()
            else:
                print(f'\tSkipping, status:{c.status}')
@forced
def kill(gpus):
    if not isinstance(gpus, tuple):
        gpus = gpus,
    gpus = set([str(i) for i in gpus])
    
    for c in cycle_c_gen():
        c_gpus = set(check_gpu(c))
        if c_gpus.intersection(set(gpus)):
            if c.status == 'running':
                c.kill()
            else:
                print(f'\tSkipping, status:{c.status}')
        else:
            print(f'\tSkipping, @ gpu {c_gpus}')

In [52]:
#export
def base_exit_handler(gpus):
    switch(gpus, 'unpause')
    
def lock(gpus, delay):
    lp = Path(cfg.GPUS.LOCK)
    #exit_handler = partial(base_exit_handler, gpus=gpus)
    switch(gpus, 'pause')
    #atexit.register(exit_handler)
    locker(gpus, delay, path=lp)

def reset():
    lp = Path(cfg.GPUS.LOCK)
    for c in cycle_c_gen():
        locked_gpus = set(str(i) for i in check_locks(lp))
        c_gpus = set(check_gpu(c))
        #print(locked_gpus, c_gpus)
        if not locked_gpus.intersection(c_gpus):
            unpause(c)
        else:
            print('locked, skip')
            pass

# Tests 

In [5]:
#export
if __name__ == '__main__':
    fire.Fire({'status':status,
               '_switch':switch,
               'reset':reset,
               'clean':clean,
               'kill':kill,
               'lock':lock
              })

ERROR: Cannot find key: -f
Usage: ipykernel_launcher.py <command>
  available commands:    status | switch | clean | kill | lock

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: 2

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
client = docker.from_env()
containers = client.containers.list(all=True)
c = containers[0]
c.name, c.status

('recursing_herschel', 'exited')

# Exports

In [7]:
!python3 extra/n2s.py tools.ipynb

Converted tools.ipynb to exp/nb_tools.py


In [11]:
!python3 exp/nb_tools.py status


Looking for *_dHPO_* containers: 
0. container	 sokolov_dHPO_May_20_13_39_11_379461 :
	sokolov_dHPO_May_20_13_39_11_379461 @ GPU['0'] @ running
1. container	 sokolov_post :
	 Skipping sokolov_post, not part of dHPO



In [20]:
!python3 exp/nb_tools.py kill force 0,1,2

0. container	 sokolov_dHPO_May_20_14_22_37_822975 :
1. container	 sokolov_post :
	 Skipping sokolov_post, not part of dHPO



In [21]:
!python3 exp/nb_tools.py clean force

0. container	 sokolov_dHPO_May_20_14_22_37_822975 :
1. container	 recursing_herschel :
	 Skipping recursing_herschel, not part of dHPO

2. container	 sokolov_post :
	 Skipping sokolov_post, not part of dHPO



In [59]:
!python3 exp/nb_tools.py reset

0. container	 sokolov_dHPO_May_20_14_49_11_381832 :
Lock /home/sokolov/work/cycler/dHPO/sync/locks/dhpo_2020_May_20_14_56_28_964324.lock is expired, deleting

	 ooo deleting/home/sokolov/work/cycler/dHPO/sync/locks/dhpo_2020_May_20_14_56_28_964324.lock
set() {'0'}
	 UNpausing sokolov_dHPO_May_20_14_49_11_381832...
	SUCCESS
1. container	 sokolov_post :
	 Skipping sokolov_post, not part of dHPO



In [6]:
!python3 exp/nb_tools.py lock 0 10

0. container	 sokolov_dHPO_May_20_15_20_24_092419 :
	sokolov_dHPO_May_20_15_20_24_092419 is using GPUS{'0'}, trying to set on "pause"
	 Pausing sokolov_dHPO_May_20_15_20_24_092419...
	SUCCESS
1. container	 sokolov_post :
	 Skipping sokolov_post, not part of dHPO

Locking "0" @ /home/sokolov/work/cycler/dHPO/sync/locks/dhpo_2020_May_20_15_26_40_054269.lock
0. container	 sokolov_dHPO_May_20_15_20_24_092419 :
	sokolov_dHPO_May_20_15_20_24_092419 is using GPUS{'0'}, trying to set on "unpause"
	 UNpausing sokolov_dHPO_May_20_15_20_24_092419...
	SUCCESS
1. container	 sokolov_post :
	 Skipping sokolov_post, not part of dHPO

